In [1]:
# %pip3 install -U altair
# %pip install -U vega vega_datasets

import altair as alt

bibliotecas = {"Altair": alt.__version__, "Vega": alt.VEGA_VERSION, "Vegalite": alt.VEGALITE_VERSION}
for i,j in bibliotecas.items():
    try:
        print(f'Versão {i:>8}: {j}')
    except:
        print(f'Biblioteca {i} não encontrada')

Versão   Altair: 5.3.0
Versão     Vega: 5
Versão Vegalite: 5.17.0


In [2]:
import altair as alt
import networkx as nx

# alt.renderers.enable('default')  # Enable the default renderer

# Data for the graph
nodes = [
    {"id": "Mecânica Clássica", "group": "Física"},
    {"id": "Mecânica Quântica", "group": "Física"},
    {"id": "Cálculo Diferencial e Integral", "group": "Matemática"},
    {"id": "Álgebra Linear", "group": "Matemática"},
    {"id": "Estatística e Probabilidade", "group": "Matemática"},
    {"id": "Geometria Diferencial", "group": "Matemática"},
    {"id": "Teoria dos Grafos", "group": "Matemática"},
    {"id": "Sim. Monte Carlo", "group": "Métodos"},
    {"id": "Dinâmica Molecular", "group": "Métodos"},
    {"id": "Integração Numérica", "group": "Métodos"},
]

edges = [
    {"source": "Mecânica Clássica", "target": "Cálculo Diferencial e Integral"},
    {"source": "Mecânica Quântica", "target": "Cálculo Diferencial e Integral"},
    {"source": "Cálculo Diferencial e Integral", "target": "Integração Numérica"},
    {"source": "Integração Numérica", "target": "Dinâmica Molecular"},
    {"source": "Álgebra Linear", "target": "Dinâmica Molecular"},
    {"source": "Estatística e Probabilidade", "target": "Sim. Monte Carlo"},
    {"source": "Estatística e Probabilidade", "target": "Dinâmica Molecular"},
    {"source": "Geometria Diferencial", "target": "Dinâmica Molecular"},
    {"source": "Teoria dos Grafos", "target": "Dinâmica Molecular"},
    {"source": "Sim. Monte Carlo", "target": "Dinâmica Molecular", "relationship": "complementar"},
]

# Criar grafo NetworkX para layout
G = nx.Graph()
for node in nodes:
    G.add_node(node['id'], group=node['group'])
for edge in edges:
    G.add_edge(edge['source'], edge['target'], relationship=edge.get('relationship', None))

# Calcular posições dos nós usando um algoritmo de layout
pos = nx.spring_layout(G, seed=42)  # Você pode experimentar com diferentes layouts

# Adicionar posições aos nós
for node in nodes:
    node['x'], node['y'] = pos[node['id']]

# Criar gráfico Altair
base = alt.Chart(alt.Data(values=nodes)).mark_circle().encode(
    x=alt.X('x:Q', axis=alt.Axis(title='Posição X')),  # Eixo X com título
    y=alt.Y('y:Q', axis=alt.Axis(title='Posição Y')),  # Eixo Y com título
    color=alt.Color('group:N', legend=None),
    tooltip=['id:N', 'group:N']
).properties(
    title='Relações Matemáticas na Dinâmica Molecular'  # Título do gráfico
).interactive()

edges_chart = alt.Chart(alt.Data(values=edges)).mark_line(point=True).encode(
    x=alt.X('source.x:Q', axis=alt.Axis(title='')),  # Eixo X sem título (compartilhado)
    y=alt.Y('source.y:Q', axis=alt.Axis(title='')),  # Eixo Y sem título (compartilhado)
    x2=alt.X2('target.x:Q'),
    y2=alt.Y2('target.y:Q'),
    detail='relationship:N',
    color=alt.condition(
        alt.datum.relationship == "complementar",
        alt.value("blue"),
        alt.value("grey"),
    ),
    # Modificar tooltip para mostrar relacionamento apenas se existir
    tooltip=[
        alt.Tooltip("source:N", title="Source"),
        alt.Tooltip("target:N", title="Target"),
        alt.Tooltip(
            field='relationship',
            type='nominal',
            title='Relationship',
        ),
    ],
).transform_filter(
    'datum.relationship'  # Filtrar relacionamentos nulos
)

chart = alt.layer(base, edges_chart).resolve_scale(
    x='shared',
    y='shared'
)

# Salvar o gráfico em um arquivo JSON
chart.save('grafo_relacoes_dinamica_molecular.json')

# Exibir o gráfico
chart

alt.LayerChart(...)

In [3]:
# Criar grafo NetworkX para layout
G = nx.Graph()
for node in nodes:
    G.add_node(node['id'], group=node['group'])
for edge in edges:
    G.add_edge(edge['source'], edge['target'], relationship=edge.get('relationship', None))

# Calcular posições dos nós usando um algoritmo de layout
# Calcular posições dos nós usando um algoritmo de layout
# pos = nx.spring_layout(G, seed=42)
# pos = nx.circular_layout(G)
pos = nx.kamada_kawai_layout(G)

# Adicionar posições aos nós
for node_id, (x, y) in pos.items():
    for node in nodes:
        if node['id'] == node_id:
            node['x'] = x
            node['y'] = y
            break

# Criar gráfico Altair (sem eixos)
base = alt.Chart(alt.Data(values=nodes)).mark_circle(size=150).encode(
    x=alt.X('x:Q', axis=None),  # Remover eixo X
    y=alt.Y('y:Q', axis=None),  # Remover eixo Y
    color=alt.Color('group:N', legend=None),
    tooltip=['id:N', 'group:N']
).properties(
    title='Relações Matemáticas na Dinâmica Molecular'
).interactive()

edges_chart = alt.Chart(alt.Data(values=edges)).mark_line(point=alt.OverlayMarkDef(size=100)).encode(
    x=alt.X('source.x:Q', axis=None),  # Coordenada x do nó de origem
    y=alt.Y('source.y:Q', axis=None),  # Coordenada y do nó de origem
    x2=alt.X2('target.x:Q'),         # Coordenada x do nó de destino (corrigido)
    y2=alt.Y2('target.y:Q'),         # Coordenada y do nó de destino (corrigido)
    detail='relationship:N',
    color=alt.condition(
        alt.datum.relationship == "complementar",
        alt.value("blue"),
        alt.value("black"),  
    ),
    tooltip=[
        alt.Tooltip("source:N", title="Source"),
        alt.Tooltip("target:N", title="Target"),
        alt.Tooltip(
            field='relationship',
            type='nominal',
            title='Relationship',
        ),
    ],
).transform_filter(
    'datum.relationship'  # Filtrar relacionamentos nulos
)

chart = alt.layer(base, edges_chart)

# Salvar o gráfico em um arquivo JSON
chart.save('grafo_relacoes_dinamica_molecular_final.json')

# Exibir o gráfico
chart

alt.LayerChart(...)

In [4]:
import altair as alt
import pandas as pd
import numpy as np

# Calcular posições para nós em um layout circular
radius = 100
n_nodes = len(nodes)
angles = np.linspace(0, 2 * np.pi, n_nodes, endpoint=False)

# Add x and y coordinates to nodes
for i, node in enumerate(nodes):
    node['x'] = radius * np.cos(angles[i])
    node['y'] = radius * np.sin(angles[i])

# Create Pandas DataFrames
df_nodes = pd.DataFrame(nodes)
df_edges = pd.DataFrame(edges)

# Create base chart for nodes
base = alt.Chart(df_nodes).mark_circle(size=150).encode(
    x=alt.X('x:Q', axis=None),  # Remover eixo X
    y=alt.Y('y:Q', axis=None),  # Remover eixo Y
    color=alt.Color('group:N', legend=None),
    tooltip=['id:N', 'group:N']
).properties(
    title='Relações Matemáticas na Dinâmica Molecular'
).interactive()

# Create chart for edges (corrected)
edges_chart = alt.Chart(df_edges).mark_line(point=alt.OverlayMarkDef(size=100)).encode(
    x=alt.X('source.x:Q', axis=None),  # Coordenada x do nó de origem
    y=alt.Y('source.y:Q', axis=None),  # Coordenada y do nó de origem
    x2=alt.X2('target.x:Q'),         # Coordenada x do nó de destino (corrigido)
    y2=alt.Y2('target.y:Q'),         # Coordenada y do nó de destino (corrigido)
    detail='relationship:N',
    color=alt.condition(
        alt.datum.relationship == 'complementar', alt.value('blue'), alt.value('grey')
    ),
    tooltip=[
        alt.Tooltip("source:N", title="Source"),
        alt.Tooltip("target:N", title="Target"),
        alt.Tooltip("relationship:N", title="Relationship") 
    ]
)

# Combine and display chart
chart = alt.layer(base, edges_chart)

chart.save('grafo_relacoes_dinamica_molecular_v8.json')

# Display the chart
chart

alt.LayerChart(...)

In [5]:
# # %pip install jupyter_contrib_nbextensions
# !jupyter contrib nbextension install --user
# !jupyter nbextension enable --py --sys-prefix widgetsnbextension


In [6]:
from pyvis.network import Network
import pandas as pd
import numpy as np

# Calcular posições para nós em um layout circular
radius = 100
n_nodes = len(nodes)
angles = np.linspace(0, 2 * np.pi, n_nodes, endpoint=False)

# Add x and y coordinates to nodes
for i, node in enumerate(nodes):
    node['x'] = radius * np.cos(angles[i])
    node['y'] = radius * np.sin(angles[i])

# Create Pandas DataFrames
df_nodes = pd.DataFrame(nodes)
df_edges = pd.DataFrame(edges)

# Create Pyvis network (with cdn_resources='remote')
net = Network(notebook=True, width="100%", height="500px", bgcolor="#222222", font_color="white", cdn_resources='remote')

# Add nodes
for _, row in df_nodes.iterrows():
    net.add_node(row['id'], label=row['id'], group=row['group'], x=row['x'], y=row['y'])

# Add edges
for _, row in df_edges.iterrows():
    net.add_edge(row['source'], row['target'], title=row.get('relationship', ''))

# Show the graph
net.show("grafo_relacoes_dinamica_molecular.html")

grafo_relacoes_dinamica_molecular.html


In [12]:
from pyvis.network import Network
import pandas as pd
import numpy as np

# Data for the graph (espaços inválidos removidos)
nodes = [
    {"id": "Mecânica Clássica", "group": "Física"},
    {"id": "Mecânica Quântica", "group": "Física"},
    {"id": "Cálculo Diferencial e Integral", "group": "Matemática"},
    {"id": "Álgebra Linear", "group": "Matemática"},
    {"id": "Estatística e Probabilidade", "group": "Matemática"},
    {"id": "Geometria Diferencial", "group": "Matemática"},
    {"id": "Teoria dos Grafos", "group": "Matemática"},
    {"id": "Sim. Monte Carlo", "group": "Métodos"},
    {"id": "Dinâmica Molecular", "group": "Métodos"},
    {"id": "Integração Numérica", "group": "Métodos"},
]

edges = [
    {"source": "Mecânica Clássica", "target": "Cálculo Diferencial e Integral"},
    {"source": "Mecânica Quântica", "target": "Cálculo Diferencial e Integral"},
    {"source": "Cálculo Diferencial e Integral", "target": "Integração Numérica"},
    {"source": "Integração Numérica", "target": "Dinâmica Molecular"},
    {"source": "Álgebra Linear", "target": "Dinâmica Molecular"},
    {"source": "Estatística e Probabilidade", "target": "Sim. Monte Carlo"},
    {"source": "Estatística e Probabilidade", "target": "Dinâmica Molecular"},
    {"source": "Geometria Diferencial", "target": "Dinâmica Molecular"},
    {"source": "Teoria dos Grafos", "target": "Dinâmica Molecular"},
    {"source": "Sim. Monte Carlo", "target": "Dinâmica Molecular", "relationship": "complementar"},
]

# Dicionário para mapear grupos a posições x
group_x_positions = {
    "Física": -150,
    "Matemática": 0,
    "Métodos": 150
}

# Raios para cada grupo (opcional, para ajustar o espaçamento)
group_radii = {
    "Física": 80,
    "Matemática": 100,
    "Métodos": 80
}

# Calcular o número de nós em cada grupo
group_counts = {}
for node in nodes:
    group = node['group']
    group_counts[group] = group_counts.get(group, 0) + 1


# Calcular os ângulos e posições x e y para cada nó
nodes_with_pos = []
for i, node in enumerate(nodes):
    group = node['group']
    radius = group_radii.get(group, 100)
    angle = (2 * np.pi / group_counts[group]) * sum(1 for n in nodes[:i] if n['group'] == group)
    node['x'] = group_x_positions[group] + radius * np.cos(angle)
    node['y'] = radius * np.sin(angle)
    nodes_with_pos.append({'id': node['id'], 'group': group, 'x': x, 'y': y})

# Create Pandas DataFrames
df_nodes = pd.DataFrame(nodes_with_pos)
df_edges = pd.DataFrame(edges)


# Create Pyvis network
net = Network(notebook=True, width="100%", height="600px", bgcolor="#ffffff", font_color="black")

# Add nodes with layered positioning
for _, row in df_nodes.iterrows():
    net.add_node(row['id'], label=row['id'], group=row['group'], x=row['x'], y=row['y'])

# Add edges with colors and relationships
for _, row in df_edges.iterrows():
    color = "blue" if row.get("relationship") == "complementar" else "grey"
    net.add_edge(row['source'], row['target'], color=color, title=row.get('relationship', ''))

# Show the graph
net.show("grafo_relacoes_dinamica_molecular.html")


grafo_relacoes_dinamica_molecular.html


In [15]:
from pyvis.network import Network
import pandas as pd
import numpy as np

# Dados dos nós e arestas (sem alterações)
nodes = [
    {"id": "Mecânica Clássica", "group": "Física"},
    {"id": "Mecânica Quântica", "group": "Física"},
    {"id": "Cálculo Diferencial e Integral", "group": "Matemática"},
    {"id": "Álgebra Linear", "group": "Matemática"},
    {"id": "Estatística e Probabilidade", "group": "Matemática"},
    {"id": "Geometria Diferencial", "group": "Matemática"},
    {"id": "Teoria dos Grafos", "group": "Matemática"},
    {"id": "Sim. Monte Carlo", "group": "Métodos"},
    {"id": "Dinâmica Molecular", "group": "Métodos"},
    {"id": "Integração Numérica", "group": "Métodos"},
]

edges = [
    {"source": "Mecânica Clássica", "target": "Cálculo Diferencial e Integral"},
    {"source": "Mecânica Quântica", "target": "Cálculo Diferencial e Integral"},
    {"source": "Cálculo Diferencial e Integral", "target": "Integração Numérica"},
    {"source": "Integração Numérica", "target": "Dinâmica Molecular"},
    {"source": "Álgebra Linear", "target": "Dinâmica Molecular"},
    {"source": "Estatística e Probabilidade", "target": "Sim. Monte Carlo"},
    {"source": "Estatística e Probabilidade", "target": "Dinâmica Molecular"},
    {"source": "Geometria Diferencial", "target": "Dinâmica Molecular"},
    {"source": "Teoria dos Grafos", "target": "Dinâmica Molecular"},
    {"source": "Sim. Monte Carlo", "target": "Dinâmica Molecular", "relationship": "complementar"},
]

# Dicionário para mapear grupos a posições x
group_x_positions = {
    "Física": -150,
    "Matemática": 0,
    "Métodos": 150
}

# Criar rede Pyvis
net = Network(notebook=True, width="100%", height="600px", bgcolor="#ffffff", font_color="black")

# Adicionar nós com posicionamento em camadas
for node in nodes:
    net.add_node(node['id'], label=node['id'], group=node['group'], 
                 x=group_x_positions[node['group']], y=np.random.randint(-100, 100))

# Adicionar arestas com cores e relacionamentos
for edge in edges:
    color = "blue" if edge.get("relationship") == "complementar" else "grey"
    net.add_edge(edge['source'], edge['target'], color=color, title=edge.get('relationship', ''))

# Exibir o gráfico
net.show("grafo_relacoes_dinamica_molecular_camadas.html")

grafo_relacoes_dinamica_molecular_camadas.html
